In [ ]:
from sklearn.model_selection import KFold,StratifiedKFold
import pandas as pd
import numpy as np


In [ ]:
train = pd.read_csv()
test = pd.read_csv()

In [ ]:
# 目标编码
# https://mp.weixin.qq.com/s/taMj-x-qLz5sw-7zld5BmA
# target encoding，可以理解为转化率，就是聚合id后对label求mean、sum、count(lable=1)/count(all)，
# 这里需要考虑到数据穿越造成线下严重过拟合的情况，一般需要通过交叉平滑处理，如划分为5折，用其中4折去统计剩下的一折，
# 然后对过大或过小的进行平滑，对测试集直接用训练集来做统计，若数据集有时间因素存在，则需要根据时间顺序来处理；
def kfold_mean(df_train, df_test, target, target_mean_list):
    folds = StratifiedKFold(n_splits=5)

    mean_of_target = df_train[target].mean()

    for fold_, (trn_idx, val_idx) in tqdm(enumerate(folds.split(df_train, y=df_train[target]))):
        tr_x = df_train.iloc[trn_idx, :]
        vl_x = df_train.iloc[val_idx, :]

        for col in target_mean_list:
            df_train.loc[vl_x.index, f'{col}_target_enc'] = vl_x[col].map(tr_x.groupby(col)[target].mean())

    for col in target_mean_list:
        df_train[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)

        df_test[f'{col}_target_enc'] = df_test[col].map(df_train.groupby(col)[f'{col}_target_enc'].mean())

        df_test[f'{col}_target_enc'].fillna(mean_of_target, inplace=True)
    return pd.concat([df_train, df_test], ignore_index=True)

feature_list =  ['HYZK', 'ZHIYE', 'ZHICHEN', 'ZHIWU', 'XUELI', 'DWJJLX', 'DWSSHY', 'GRZHZT']
data = pd.concat([train, test], ignore_index=True)
data = kfold_mean(data[~data['label'].isna()], data[data['label'].isna()],'label',feature_list)
print(data)
